## Pulling data from Firebase
* This file will not run due to no prvkey.json

In [ ]:
#!pip3 install firebase_admin

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json
import csv as csv

In [3]:
def get_classes(db,num,clss,rooms,all_data,lounge):
        err=0
        loop=0
        quit = False
        while len(rooms)>0 and not quit:
                
                if  loop<num:
                        loop+=1
                        room=rooms[-1]
                        print('open',room)
                        for l in lounge:
                                if l.id==room:
                                        
                                        main_class=l.to_dict()['classID']
                                        if main_class not in all_data:
                                                all_data[main_class]=dict()
                                        if room not in all_data[main_class]:
                                                all_data[main_class][room]=dict()
                                        all_data[main_class][room]['document']=l.to_dict()['docID']
                                        all_data[main_class][room]['users']=[]
                                        all_data[main_class][room]['creater']=clss[main_class]['userID']
                                        break

                        rooms.pop()
                        logs=  db.collection("rooms").document(room).collections()
                        for log in logs:
                                try:
                                        if log.id=='logs':
                                                for lo in log.stream():
                                                                
                                                        user={}
                                                        user['user_id']=lo.id
                                                        user['user_name']=lo.to_dict()['userName']
                                                        if 'email' in lo.to_dict():
                                                                user['email']=lo.to_dict()['email']
                                                        user['logs']=[]
                                                        user['hands']=0
                                                        try:
                                                                for l in log.document(lo.id).collections():
                                                           
                                                                        if l.id=='steps':
                                                                                        
                                                                                for s in l.stream():
                                                                                        if 'enter' and 'time' in s.to_dict():
                                                                                                user['logs'].append({'enter':s.to_dict()['enter'],'leave':s.to_dict()['leave'],
                                                                                                                'duration':s.to_dict()['duration'],'time':s.to_dict()['time'].astimezone().strftime("%d/%m/%Y, %H:%M:%S")})

                                                                        elif l.id=='hands':
                                                                                for s in l.stream():
                                                                                        user['hands']+=1
                                                        except:
                                                                pass
                                                                                
                                                        
                                                                
                                                        all_data[main_class][room]['users'].append(user)      
                                except:
                                        print("err")
                                        err+=1  
                        print(room)
                else:
                        quit=True


In [4]:
def get_5_class_in_a_row(i,u_n,clss,rooms,all_data):
    
    cred = credentials.Certificate('pvkey.json')
    if i==0:
        print("______________",'batdau',"__________________")
        app = firebase_admin.initialize_app(cred)
    else:
        print("______________",u_n[i-1],"__________________")
        app = firebase_admin.initialize_app(cred,name=u_n[i-1])

    db = firestore.client()
    lounge= db.collection("rooms").stream()
    
    
    get_classes(db,3,clss,rooms,all_data,lounge)
    
    with open('new_raw_data.json','w') as datata:
        print(len(all_data))
        json.dump(all_data, datata)
def get_data(clss,rooms,all_data):
    # Loop hole
    # Firebase limit user access to database -> fake user
    u_n=['hung20gg','hungg20','mpqh','nthh','meo','meomeomeo','kkk','lamng','lampx84','lll','lil_nas_x',
     'jenny','lalisa','rose','jisoo','tueanh04','thuan','ngoccha','taylor','swift','nttuan7','hung19gg','lil_toilet',
     'meomeomeomeo','tpa','cnpa','ntpt','ntp','mpn','1shit','exit','kakaka','loc','end','V','Suga','RM','Jungkook']
    for i in range(len(u_n)):
        
        get_5_class_in_a_row(i,u_n,clss,rooms,all_data)

In [6]:

cl=open('classes.json')

have_u=0
rooms=[]
f_o=False
try:
    f = open('new_raw_data.json')
    all_data = json.load(f)
    f_o=True
except:
    all_data=dict()
clss =json.load(cl)
    # Iterating through the json
    # list
take_class=['Yorao','X0bMT','ROmwS','L71ar']
not_a_class=['description','subjectID','memberList','userID','name','classID','rooms','order']
done=[]
for t in all_data:
    done.append(t)
for clas in clss:
    if clas in take_class:
        
        for room in clss[clas]:
            
            if room not in not_a_class and room not in done and len(room)>=5 and len(room)<=7:
                
                rooms.append(room)

# Closing file
print(f_o)
if f_o:
    f.close()
cl.close()

True


In [ ]:
print(rooms)

['viL1Ch', '7K9gs2', 'tlrxeV', 'UI31Yv', 'eZ5VOh', 'l03lkE', 'nA0T4z', 'NGaQ4z', 'ATL2kf', 'yBsdsk', '2ooRiT', 'UbFrDI', 'aL8zwu', 'XxYejV', '30gSeC', 'CdQCHA', 'SyNgFE', 'i9xKXG', 'Wn4FDB', 'pPyNHQ', 'Qq1Mt1', 'EL3lS3', 'MRTUkV', 'qPnLTc', 'qk4RZv', 'EkfcXS', 'UxAiZa', 'nThQRl', 'EDLslR', 'U0z5ZC', 'ta1YwR', 'TeN0BH', '87FIFJ', '0xYhGf', 'EcP9hj']
